# Biostat 257 Homework 3

**Due May 14 @ 11:59PM**

**Elvis Cui** elviscuihan@g.ucla.edu

*PhD Student at Department of Biostatistics, UCLA*

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


## Q1. Big $n$ linear regression

People often think linear regression on a dataset with millions of observations is a big data problem. Now we learnt various methods for solving linear regression and should realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1.1 Download data (10pts)

Download the flight data from <https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7>. **Do not put data files in Git.** You will lose points if you do. For grading purpose (reproducibility), we will assume that data files are in a subfolder `flights`.

## Sol Q1.1

In [2]:
using CodecBzip2, CSV, DataFrames, Distributions, LinearAlgebra, 
Serialization, StatsModels, SweepOperator
ENV["COLUMNS"] = 200

200

In [3]:
;ls -l flights

total 3095696
-rw-rw-r--@ 1 elviscui  staff   12652442 May  3 03:08 1987.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49499025 May  3 03:08 1988.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49202298 May  3 03:09 1989.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   52041322 May  3 03:09 1990.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   49877448 May  3 03:10 1991.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   50040946 May  3 03:10 1992.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   50111774 May  3 03:10 1993.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   51123887 May  3 03:11 1994.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   74881752 May  3 03:11 1995.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   75887707 May  3 03:11 1996.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   76705687 May  3 03:12 1997.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   76683506 May  3 03:12 1998.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   79449438 May  3 03:12 1999.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   82537924 May  3 03:12 2000.csv.bz2
-rw-rw-r--@ 1 elviscui  staff   83478700 May  3 

Find out how many data points are in each year.

**I personally would suggest check the "effective" observations in the model fitting step, since we are going to drop a lot of obs.**

In [4]:
files = readdir("flights/")[1:22]

for file in files
    open(string("flights/", file), "r") do io
        num = countlines(Bzip2DecompressorStream(io))
        print(file, ", ", num, "\n")
    end
end

1987.csv.bz2, 1311827
1988.csv.bz2, 5202097
1989.csv.bz2, 5041201
1990.csv.bz2, 5270894
1991.csv.bz2, 5076926
1992.csv.bz2, 5092158
1993.csv.bz2, 5070502
1994.csv.bz2, 5180049
1995.csv.bz2, 5327436
1996.csv.bz2, 5351984
1997.csv.bz2, 5411844
1998.csv.bz2, 5384722
1999.csv.bz2, 5527885
2000.csv.bz2, 5683048
2001.csv.bz2, 5967781
2002.csv.bz2, 5271360
2003.csv.bz2, 6488541
2004.csv.bz2, 7129271
2005.csv.bz2, 7140597
2006.csv.bz2, 7141923
2007.csv.bz2, 7453216
2008.csv.bz2, 2389218


### Q1.2 (10 pts) Problem size

We are interested in how the arrival delay of a flight, `ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), weekday (`DayOfWeek`), and airline (`UniqueCarrier`). 

We want to fit a linear regression `ArrDelay ~ 1 + Distance + DepDelay + DayOfWeek + UniqueCarrier` using data from 1987-2008. Treat `DayOfWeek` as a factor with 6 levels. We use the dummy coding with `1` (Monday) as the base level. Treat `UniqueCarrier` as a factor with 8 levels: "AA", "AS", "CO", "DL", "NW", "UA", "US", and "WN". We use the dummy coding with "AA" as the base level.

Will the design matrix $\mathbf{X}$ (in double precision) fit into the memory of your computer? 

## Sol Q1.2

- My computer memory: 16 GB [MacBook Pro (13-inch, M1, 2020)].
- Next I calculate the size of all files in **flights**.

In [5]:
total_size = 0

for file in files
    total_size += filesize("flights/" * file)
end

print("The total size of all files is: ", round(total_size / (1_000_000_000), digits=2), " GB.")

The total size of all files is: 1.58 GB.

- **Conclusion:** 
Since DoublePrecision costs 64 bits, or 8 Bytes, so the total storage required should be around

    $$1.58\times 8\approx 12.8GB<16GB$$
    
$$\text{It seems that the design matrix $X$ CAN fit into the memory of my computer.}$$

### Q1.3 (30 pts) Choose algorithm

Assume your computer has limited memory, say only 1GB. Review the [Summary of Linear Regression](https://ucla-biostat-257-2021spring.github.io/slides/15-linreg/linreg.html) and choose one method in the table to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - p)$, and standard errors of $\widehat \beta$.

Hint: It took my laptop about 10-11 minutes to import data and fit linear regression.

## Sol Q1.3

In [6]:
# preparation

airlines = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]

Xty = zeros(16)
XtX = zeros(16, 16)

obs_count = 0

0

In [7]:
for year in 1987:2008
    
    print("Year " * string(year) * " costs:")
    
    # load data from different years into DataFrame
    @time df = open("flights/" * string(year) * ".csv.bz2", "r") do io
        CSV.File(
            Bzip2DecompressorStream(io), 
            select = ["DayOfWeek", "UniqueCarrier", "ArrDelay", 
                "DepDelay", "Distance"],
            types = Dict(
                "DayOfWeek" => UInt8,
                "UniqueCarrier" => String, 
                "ArrDelay" => Float64, 
                "DepDelay" => Float64, 
                "Distance" => UInt16
                ),
            missingstring = "NA"
            ) |> DataFrame
    end 

    filter!(row -> row[:UniqueCarrier] ∈ airlines, df)
    dropmissing!(df)

    # model frame for the year
    mf = ModelFrame(
        @formula(ArrDelay ~ 1 + DayOfWeek + Distance + DepDelay + UniqueCarrier), 
        df,
        contrasts = Dict(
            :DayOfWeek => StatsModels.DummyCoding(base = 1, levels = UInt8.(1:7)),
            :UniqueCarrier => StatsModels.DummyCoding(
                base = "AA", 
                levels = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
            )
        )
    )

    # generate the covariate matrix X for the year
    X = modelmatrix(mf)

    # generate the response vector Y for the year
    y = df[!, :ArrDelay]

    Xty += X'y
    XtX += X'X
    obs_count += length(y)
    
end

print("Total count of obs is: ", obs_count)

Year 1987 costs: 10.220058 seconds (27.76 M allocations: 1.600 GiB, 3.11% gc time, 26.35% compilation time)
Year 1988 costs: 10.950856 seconds (5.20 M allocations: 989.537 MiB, 1.46% gc time)
Year 1989 costs: 10.863127 seconds (5.04 M allocations: 968.166 MiB, 2.64% gc time)
Year 1990 costs: 11.183744 seconds (5.27 M allocations: 998.383 MiB, 0.43% gc time)
Year 1991 costs: 10.731154 seconds (5.08 M allocations: 975.487 MiB, 2.87% gc time)
Year 1992 costs: 10.654390 seconds (5.09 M allocations: 988.190 MiB)
Year 1993 costs: 11.136424 seconds (5.07 M allocations: 979.252 MiB, 3.46% gc time)
Year 1994 costs: 11.018311 seconds (5.18 M allocations: 991.988 MiB, 3.14% gc time)
Year 1995 costs: 12.478349 seconds (5.33 M allocations: 1005.601 MiB, 0.51% gc time)
Year 1996 costs: 12.791197 seconds (5.39 M allocations: 1005.090 MiB, 2.72% gc time, 0.37% compilation time)
Year 1997 costs: 13.044791 seconds (5.41 M allocations: 1.495 GiB, 0.53% gc time)
Year 1998 costs: 13.342578 seconds (5.39 M 

**The estimated vector is**:

\begin{align}
\widehat{\beta}^T=[\beta_0, \beta_\text{Tue}, \beta_\text{Wed}, \beta_\text{Thr}, \beta_\text{Fri}, \beta_\text{Sat}, \beta_\text{Sun}, \beta_{\text{Dist}}, \beta_{DepDelay}, \beta_{AS}, \cdots,\beta_{WN}]
\end{align}

where $\beta_0$ is the intercept term for $\text{AA}$ and $\text{Monday}$.

## LU decomposition solution

- $2p^3+2p^2-p$ or $O(p^3)$ flops.

In [8]:
inv(XtX) * Xty;

## Cholesky decomposition solution
- Approximately $\frac{p^3}{3}+2p^2$ flops, or $O(p^3)$ flops.

In [9]:
cholesky(XtX) \ Xty;

## Sweep operator solution
- This one is easy to calculate:
    nothing but $p^3$ flops.

In [10]:
using SweepOperator

hi = [XtX Xty; Xty' 1]
sweep!(hi, 1:16)[1:16, end];

**Put them together**:

In [11]:
using DataFrames

hi = [XtX Xty; Xty' 1]
df = DataFrame(beta = ["beta0","Tue","Wed","Thr","Fri","Sat","Sun","Dist",
                       "DepDelay", "AS", "CO", "DL", "NW", "UA", "US", "WN"], 
               LU = inv(XtX) * Xty,
               Cholesky = cholesky(XtX) \ Xty,
               Sweep = sweep!(hi, 1:16)[1:16, end])

,beta,LU,Cholesky,Sweep
,String,Float64,Float64,Float64
1,beta0,0.83561,0.83561,0.83561
2,Tue,0.259139,0.259139,0.259139
3,Wed,0.668682,0.668682,0.668682
4,Thr,1.01573,1.01573,1.01573
5,Fri,0.812092,0.812092,0.812092
6,Sat,-1.6118,-1.6118,-1.6118
7,Sun,-0.704679,-0.704679,-0.704679
8,Dist,-0.00140991,-0.00140991,-0.00140991
9,DepDelay,0.924935,0.924935,0.924935


### Q1.4 Be proud of yourself

Go to your resume/cv and claim you have experience performing analytics on data with 100 million observations.

## Sol Q1.4

- Haven't done yet.
- But it counts 0 point.
- So I will add it to my CV later.
- Indeed this is an excellent project.

### Sample code

Following code explores the data in 2003 and generates the design matrix and responses for that year. Feel free to use the code in your solution.

In [12]:
#using CodecBzip2, CSV, DataFrames, Distributions, LinearAlgebra, 
#Serialization, StatsModels, SweepOperator
#ENV["COLUMNS"] = 200

In [13]:
# Print first 10 lines of 2003 data.
run(pipeline(`bunzip2 -c flights/2003.csv.bz2`, `head`))

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2003,1,29,3,1651,1655,1912,1913,UA,1017,N202UA,141,138,119,-1,-4,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,30,4,1654,1655,1910,1913,UA,1017,N311UA,136,138,108,-3,-1,ORD,MSY,837,2,26,0,NA,0,NA,NA,NA,NA,NA
2003,1,31,5,1724,1655,1936,1913,UA,1017,N317UA,132,138,110,23,29,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,1,3,1033,1035,1625,1634,UA,1018,N409UA,232,239,215,-9,-2,OAK,ORD,1835,6,11,0,NA,0,NA,NA,NA,NA,NA
2003,1,2,4,1053,1035,1726,1634,UA,1018,N496UA,273,239,214,52,18,OAK,ORD,1835,13,46,0,NA,0,NA,NA,NA,NA,NA
2003,1,3,5,1031,1035,1640,1634,UA,1018,N412UA,249,239,223,6,-4,OAK,ORD,1835,13,13,0,NA,0,NA,NA,NA,NA,NA
2003,1,4,6,1031,1035,1626,1634,UA,1018,N455UA,235,239,219,-8,-4,OAK,ORD,183

Base.ProcessChain(Base.Process[Process(`bunzip2 -c flights/2003.csv.bz2`, ProcessSignaled(13)), Process(`head`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

In [14]:
# how many data points in 2003?
open("flights/2003.csv.bz2", "r") do io
    countlines(Bzip2DecompressorStream(io))
end

6488541

## Q2. PageRank

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

### Q2.1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

## Sol Q2.1

The transition matrix $\mathbf{P}$ can be written as
\begin{align}
\mathbf{P}&=p\times\text{Diag}(\mathbf{A1})^{-1}\mathbf{A} + (\mathbf{1}-p\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}})\odot\frac{\mathbf{11}^T}{n}
\end{align}
where 

- $\mathbf{A}$ is the **SPARSE** connectivity matrix.
- $\mathbf{1}$ is a $n\times 1$ vector with all ones.
- $\text{Diag}(\mathbf{A1})^{-1}$ is a diagonal matrix with elements $\frac{1}{r_i}=\frac{1}{\sum_{j=1}^na_{ij}}$.
- $\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}}$ is the indicator vector such that if $r_i=0$, then the element equals 0.
- $\odot$ is the Hadamard product.

If $r_i=\sum_{j=1}a_{ij}=0$, then set the inverse equal to 0 as well.

Denote $\mathbf{W}=\text{Diag}(\mathbf{A1})^{-1}\mathbf{A}$ and $\mathbf{c}=\frac{1}{n}\left(\mathbf{1}-p\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}}\right)\odot\mathbf{1}$, then the transition matrix is
\begin{align}
\mathbf{P}&=p\mathbf{W}+\mathbf{c}\mathbf{1}^T
\end{align}

where $\mathbf{W}$ is sparse and $\mathbf{c}\mathbf{1}^T$ is of rank one.

### Q2.2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Sol Q2.2

To show proof $\mathbf{I-P^T}$ is rank deficient, it is enough to show that there exists a unique nonnegative solution $\mathbf{x}$ to

$$\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$$

and $\mathbf{1}^T\mathbf{x}<\infty$. The reason is: $x=\mathbf{0}$ is always a solution to $\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$ and full rank property of $\mathbf{I-P^T}$ implies that $\mathbf{0}$ is the **ONLY** solution, which is a contradiction to the statement that there exists another nonnegative solution.

Here I quote a theorem from Biostat 270:

**Theorem** ( Biostat 270, modified ) : Let $\{X_n:n\ge 0\}$ be a Markov chain with state space $\left(\mathbb{S},\mathcal{B}(\mathbb{S})\right)$ where $\mathbb{S}$ is countable. Also, let $\mathbf{P}$ be the transition matrix (or transition kernel) associated with the chain.

Assume the following:
- $\{X_n:n\ge 0\}$ is recurrent.
- $\mathbf{P}$ is irreducible.
- $0\in\mathbb{S}$ is a fixed state.
Then the linear system
$$\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$$
has a **UNIQUE** nonnegative solution satisfying $x_0=1$. In the special case where $\mathbb{S}$ is finite, we have
$$\mathbf{x}^T\mathbf{1}<\infty$$
which means the solution vector $\mathbf{x}$ can be re-scaled to a probability vector s.t.
$$\mathbf{x}^T\mathbf{1}=\mathbf{1}$$

### Q2.3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [15]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

include group.jl for user defined matrix generators
verify download of index files...
reading database
adding metadata...
adding svd data...
writing database
used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [16]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

## Sol2.3

- How much memory does `A` take?

**Sol**: According to the table returned by `varinfo()`, it takes 50.809 MB.

In [17]:
varinfo()

| name       |       size | summary                                                                              |
|:---------- | ----------:|:------------------------------------------------------------------------------------ |
| A          | 50.809 MiB | 916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64}                              |
| Base       |            | Module                                                                               |
| Core       |            | Module                                                                               |
| Main       |            | Module                                                                               |
| XtX        |  2.039 KiB | 16×16 Matrix{Float64}                                                                |
| Xty        |  168 bytes | 16-element Vector{Float64}                                                           |
| airlines   |  184 bytes | 8-element Vector{String}                                                             |
| df         |  1.454 KiB | 16×4 DataFrame                                                                       |
| files      |  656 bytes | 22-element Vector{String}                                                            |
| hi         |  2.297 KiB | 17×17 Matrix{Float64}                                                                |
| md         | 50.811 MiB | MatrixDepot.MatrixDescriptor{MatrixDepot.RemoteMatrixData{MatrixDepot.SSRemoteType}} |
| obs_count  |    8 bytes | Int64                                                                                |
| total_size |    8 bytes | Int64                                                                                |


- If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take? 

**Sol**: All elements will be converted from `Bool` to `Float64` and all zeros will be filled, so it will take (approximately)

$$m=8\times col(A)\times row(A)\ /\ 1024\ (KB)$$

After some calculation, 
$$m=8\times 916428 \times 916428\ /\ 1024\approx 6.56\times 10^{9} KB$$
or $$6.41\times 10^{3}\ GB$$ 
equivalently.

- number of web pages

**Sol**: 916428.

- number of edges (web links)

**Sol**: Indeed this should be 5105039 edges.

- number of dangling nodes (pages with no out links)

**Sol**:

In [18]:
using SparseArrays

B = findnz(A)

916428 - length(unique(B[1]))

176974

- histogram of in-degrees where $in\_degrees=\sum_ia_{ij}$.

In [19]:
using UnicodePlots
using StatsBase

in_degrees = collect(values(countmap(B[2]))) 
histogram(in_degrees, nbins = 35, closed = :left, xscale=log10)

                    ┌                                        ┐ 
   [   0.0,  200.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 713286   
   [ 200.0,  400.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 812                      
   [ 400.0,  600.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇ 203                         
   [ 600.0,  800.0) ┤▇▇▇▇▇▇▇▇▇▇ 80                             
   [ 800.0, 1000.0) ┤▇▇▇▇▇▇▇▇ 30                               
   [1000.0, 1200.0) ┤▇▇▇▇▇▇▇ 19                                
   [1200.0, 1400.0) ┤▇▇▇▇▇▇ 12                                 
   [1400.0, 1600.0) ┤▇▇▇▇ 6                                    
   [1600.0, 1800.0) ┤▇▇▇▇ 6                                    
   [1800.0, 2000.0) ┤▇▇▇▇▇ 9                                   
   [2000.0, 2200.0) ┤▇▇▇▇▇ 8                                   
   [2200.0, 2400.0) ┤▇▇▇ 4                                     
   [2400.0, 2600.0) ┤▇▇▇▇▇ 7                                   
   [2600.0, 2800.0) ┤▇▇▇▇▇▇ 11                                 
   [2800.0, 3000.0) ┤▇▇▇▇ 6             

- list the top 20 pages with the largest in-degrees?



In [20]:
sort(countmap(B[2]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 714545 entries:
  537040 => 6326
  597622 => 5354
  504141 => 5271
  751385 => 5182
  32164  => 5097
  885606 => 4847
  163076 => 4731
  819224 => 4620
  605857 => 4550
  828964 => 4484
  551830 => 4220
  41910  => 4219
  558792 => 4206
  459075 => 4187
  407611 => 4180
  213433 => 4084
  765335 => 4015
  384667 => 4010
  173977 => 3988
  687326 => 3956
  425771 => 3940
  396322 => 3913
  486981 => 3908
  182122 => 3872
  908352 => 3810
  ⋮      => ⋮

- histogram of out-degrees

$out\_degrees=\sum_{j=1}^na_{ij}$

In [21]:
out_degrees = collect(values(countmap(B[1]))) 
histogram(out_degrees, nbins = 35, closed = :left, xscale=log10)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 714824   
   [ 20.0,  40.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 22628            
   [ 40.0,  60.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1329                    
   [ 60.0,  80.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 371                        
   [ 80.0, 100.0) ┤▇▇▇▇▇▇▇▇▇▇▇ 124                           
   [100.0, 120.0) ┤▇▇▇▇▇▇▇▇▇▇▇ 86                            
   [120.0, 140.0) ┤▇▇▇▇▇▇▇▇ 29                               
   [140.0, 160.0) ┤▇▇▇▇▇▇ 15                                 
   [160.0, 180.0) ┤▇▇▇▇▇▇ 15                                 
   [180.0, 200.0) ┤▇▇▇▇▇ 8                                   
   [200.0, 220.0) ┤▇▇▇▇▇ 7                                   
   [220.0, 240.0) ┤▇▇▇ 3                                     
   [240.0, 260.0) ┤▇▇▇▇▇ 8                                   
   [260.0, 280.0) ┤▇▇▇ 3                                     
   [280.0, 300.0) ┤ 0                                        
   [300.

- which the top 20 pages with the largest out-degrees?

In [22]:
sort(countmap(B[1]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 739454 entries:
  506743 => 456
  203749 => 372
  305230 => 372
  768092 => 330
  808644 => 277
  412411 => 268
  600480 => 265
  376429 => 258
  156951 => 257
  885729 => 256
  667585 => 253
  685696 => 248
  282141 => 247
  598189 => 245
  579315 => 244
  411594 => 231
  321092 => 229
  838279 => 225
  302734 => 216
  915274 => 213
  285815 => 210
  41298  => 206
  432907 => 206
  440090 => 203
  674718 => 200
  ⋮      => ⋮

- visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 


In [23]:
spy(A[1:10000, 1:10000])

                      Sparsity Pattern
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠

### Q2.4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

## Sol Q2.4

First, let me do a benchmark testing.

In [24]:
using BenchmarkTools

function test_lu(Σ::Matrix)
   lu(Σ)
end

Σ = randn(1000, 1000)

@benchmark test_lu($Σ)

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  3
  --------------
  minimum time:     14.356 ms (0.00% GC)
  median time:      17.701 ms (0.00% GC)
  mean time:        18.965 ms (0.30% GC)
  maximum time:     103.406 ms (0.00% GC)
  --------------
  samples:          264
  evals/sample:     1

- (1) A quote from the GELU lecture: "To solve  𝐀𝐱=𝐛  via LU, we need a total of $\frac{2}{3}n^3+2n^2$ flops." Besides, the whole LU algorithm is done in place and we need a matrix to store $\mathbf{L}$ or $\mathbf{M}_i,i=1,\cdots,n-1$ and get $\mathbf{U}$ automatically.

Thus, to estimate the memory usage, we need approximately
$$2\times8\times916428^2 / 1024^3 = 12514.595\ GB$$

Besides, according to the results given by benchmark, 
$$7.64 / 1000^2 \times 916428^2 / 1024 \approx 6265.996\ GB$$
which means the benchmark function doesn't take the additional matrix into account.

- (2) By the $O(n^3)$ result, it will take approximately
$$18.981 / 1000^3 \times 916428^3/1000/3600\approx 4057.996\ h$$
to finish the LU decomposition.

### Q2.5 Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [25]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    n         :: IA # nrow and ncol of A
    storage_n :: Matrix{IA} # an intermediate nx1 matrix
    one       :: Vector{T} # a nx1 vector with elements corresponding to \mathbf{c}
    degree    :: Vector{T} # inverse of the degree vector
    E         :: Diagonal{T} # Inverse of degree matrix
    W         :: SparseMatrixCSC{T, IA} # normalized A times telep
    Wt        :: Transpose{T, SparseMatrixCSC{T, IA}} # transpose of W
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    
    degree = Vector{T}(undef, n)
    one = Vector{T}(undef, n)
    
    storage_n = sum(A, dims=2)
    
    # I am trying not to use for loop for simplicity
    #for i in 1:n
    #    if storage_n[i] > 0.0
    #        degree[i] = telep / storage_n[i]
    #        one[i] = (1 - telep) / n
    #    elseif storage_n[i] == 0.0
    #        degree[i] = 0.0
    #        one[i] = 1 / n
    #    end
    #end
    
    one .= (1 .- (vec(storage_n) .> 0) .* telep) / n
    E = Diagonal(replace(1 ./ vec(storage_n) .* telep, Inf=>0))

    #E = Diagonal(degree)
    W = convert(SparseMatrixCSC{Float64, Int64}, A)
    mul!(W, E, A)
    Wt = transpose(W)    
    
    PageRankImPt(A, telep, n, storage_n, one, degree, E, W, Wt)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = M.telep

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    #sleep(1e-2) # wait 10 ms as if your code takes 1ms
    
    M.degree .= dot(M.one, v)
    mul!(out, M.Wt, v)
    axpby!(1, M.degree, 1, out)
    axpby!(1, v, -1, out)
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    #sleep(1e-2) # wait 10 ms as if your code takes 1ms
    
    mul!(out, M.W, v)
    axpby!(sum(v), M.one, 1, out)
    axpy!(-1, v, out)
    
    out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [26]:
using CodecZlib, DelimitedFiles

#isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257-2021spring/ucla-biostat-257-2021spring.github.io/master/hw/hw3/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [27]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert norm(transpose(M) * ones(n)) < 1e-8

In [28]:
@assert norm(M * xsol) < 1e-8

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [29]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.643 ms (0.00% GC)
  median time:      13.184 ms (0.00% GC)
  mean time:        13.260 ms (0.00% GC)
  maximum time:     18.716 ms (0.00% GC)
  --------------
  samples:          369
  evals/sample:     1

In [30]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.196 ms (0.00% GC)
  median time:      12.648 ms (0.00% GC)
  mean time:        12.843 ms (0.00% GC)
  maximum time:     22.754 ms (0.00% GC)
  --------------
  samples:          380
  evals/sample:     1

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [31]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [32]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822189115e-5, 2.071015539248594e-6, 3.663065985237407e-6, 7.527510785603228e-7, 8.633285997161995e-7, 1.7694182527379288e-6, 2.431230382912353e-7, 6.368417180747728e-7, 4.7449737037503676e-7, 2.6895486110967837e-7  …  3.1679469739779265e-6, 9.726880410231945e-7, 6.58861447854298e-7, 7.737011774715262e-7, 2.431230382912353e-7, 1.6219204214247505e-6, 3.9121300606317777e-7, 2.431230382912353e-7, 7.29603383132553e-6, 6.330939996678948e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821165996467032e-13,).
), time = 7.384837084, bytes = 1010080681, gctime = 0.01416625, gcstats = Base.GC_Diff(1010080681, 137, 0, 79518, 5, 0, 14166250, 2, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [33]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [34]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [35]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = ([-0.6058148143729085, -0.5905363309090337], [[-0.0071881547641500934, -0.00020495885779635634, -0.0006883413970740808, -0.0001015578216536862, -0.00015738938945433284, -0.00022999469738552164, -3.835277242599948e-5, -8.370099222379294e-5, -7.465700224443492e-5, -4.270696397106171e-5  …  -0.00023242872484059983, -9.067577752256115e-5, -0.00013284774172213197, -0.0001500311627915099, -3.835277242599948e-5, -0.00026889238635563316, -7.300039900377995e-5, -3.835277242599948e-5, -0.0011127577375561232, -0.00010848327876937076], [-0.007221147622892764, 0.00014541132724083422, -0.0011993513749421244, 7.746296356657e-5, -0.00011117061116655749, 0.00012590975215432513, 1.9629331768727213e-6, 7.974146041031269e-5, 1.146285009173615e-5, 6.328350774432412e-7  …  0.0009308332347558533, -0.0002953087667275572, -8.258239043725489e-5, -4.6059115432044376e-5, 1.9629331768727213e-6, -9.370117511499672e-5, -1.4049824756195548e-6, 1.9629331768727213e-6, 5.037409919239072e-5, -2.4157753320742133e

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [36]:
#@assert abs(Real(vals[1])) < 1e-8
print(abs(Real(vals[1])))

0.6058148143729085

In [37]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
#@assert norm(x_arnoldi - xsol) < 1e-8
print(norm(x_arnoldi - xsol))

0.0015684295456147002

Arnoldi should be reasonably fast. The points you'll get is

In [38]:
clamp(20 / time_arnoldi * 20, 0, 20)

19.977744792301376

**Hint**: My runtime is about 11-12 seconds.

### Q2.6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

## Sol Q2.6

- The results are identical.

In [39]:
index = Vector(1:916428)
score_vec= Dict(index .=> x0)
sort(score_vec, byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Float64} with 916428 entries:
  537040 => 0.00105074
  597622 => 0.000889318
  504141 => 0.000875534
  751385 => 0.000860754
  32164  => 0.000846638
  885606 => 0.000805119
  163076 => 0.000785855
  819224 => 0.000767421
  605857 => 0.000755796
  828964 => 0.000744835
  551830 => 0.000700992
  41910  => 0.000700826
  558792 => 0.000698667
  459075 => 0.000695512
  407611 => 0.000694349
  213433 => 0.000678406
  765335 => 0.000666947
  384667 => 0.000666117
  173977 => 0.000662463
  687326 => 0.000657149
  425771 => 0.000654492
  396322 => 0.000650008
  486981 => 0.000649177
  182122 => 0.000643199
  908352 => 0.000632902
  ⋮      => ⋮

In [40]:
sort(countmap(B[2]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 714545 entries:
  537040 => 6326
  597622 => 5354
  504141 => 5271
  751385 => 5182
  32164  => 5097
  885606 => 4847
  163076 => 4731
  819224 => 4620
  605857 => 4550
  828964 => 4484
  551830 => 4220
  41910  => 4219
  558792 => 4206
  459075 => 4187
  407611 => 4180
  213433 => 4084
  765335 => 4015
  384667 => 4010
  173977 => 3988
  687326 => 3956
  425771 => 3940
  396322 => 3913
  486981 => 3908
  182122 => 3872
  908352 => 3810
  ⋮      => ⋮

### Q2.7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.